# T02

Tomás González Villarroel

In [13]:
import pandas as pd
import numpy as np

print(3/8)
-1 * (3/8 * np.log2(3/8) + 5/8 * np.log2(5/8))

3/32 / 0.109375

1/64 /* 32/5

0.375


0.8571428571428571

## Definición de clases

In [2]:
class Node:

    def __init__(self, data, target, depth=0, division=None, value=None):
        self.division = division 
        self.value = value 
        self.data = data
        self.target = target
        self.children = None
        self.answer = None
        self.depth = depth
        self.es_hoja = False
        self.es_categorico = True
        self.bin = None
        self.predict_data = None


    def calculate_entropy(self):
        """
        Calcula la entropía del nodo y la retorna
        """
        grouped = self.data[self.target].value_counts()
        self.answer = grouped.idxmax()

        total = len(self.data)
        entropy = -1 * np.sum([g/total * np.log2(g/total) for g in grouped])
        return entropy

    def get_best_division(self, strategy='median', cut_type='traditional'):
        """
        Retorna el nombre de la/s columna/s con las cuales hará la división el nodo.
        """
        gains = {}
        tipos = {}
        bins = {}
        entropy = self.calculate_entropy()
        if entropy == 0:
            self.es_hoja = True
            return False
        total_cols = self.data.drop(self.target, axis=1).columns
        total_data = len(self.data)

        if not total_data:
            self.es_hoja = True
            return None
        nodo_posible = Node(None, self.target, depth=self.depth + 1)
        
        if cut_type == 'traditional':
            for colname in total_cols:
                col = self.data[colname]
                tipos[colname] = True
                if col.dtype.name != 'object':
                    col, bin = self.bining(col, strategy)
                    tipos[colname] = False
                    bins[colname] = bin
                values = col.value_counts().to_dict()
                gains[colname] = entropy
                split_info = 0
                for v in values:
                    new_data = self.data[col == v]
                    if len(new_data) == 0:
                        continue
                    if col.dtype.name == 'object':
                        new_data = new_data.drop(col.name, axis=1)
                    nodo_posible.data = new_data
                    nodo_posible.value = v
                    gains[colname] -= len(new_data)/total_data * nodo_posible.calculate_entropy()
                    split_info -= values[v]/total_data * np.log2(values[v]/total_data)
                if split_info != 0:
                    gains[colname] /= split_info

            result = max(gains, key=gains.get)
            self.division = result
            self.es_categorico = tipos[result]
            if not self.es_categorico:
                self.bin = bins[result]
            return result
        else:
            if len(total_cols) < 2:
                self.es_hoja = True
                return None
            
            values = []
            for col in total_cols:
                col = self.data[col]
                tipos[col.name] = True
                if col.dtype.name != 'object':
                    col, bin = self.bining(col, strategy)
                    tipos[col.name] = False
                    bins[col.name] = bin
                values.append(col.value_counts().to_dict())

            for i in range(len(total_cols)):
                for j in range(i + 1, len(total_cols)):
                    col1, col2 = self.data[total_cols[i]], self.data[total_cols[j]]
                    gains[(col1.name, col2.name)] = entropy
                    split_info = 0
                    for v1 in values[i]:
                        for v2 in values[j]:
                            new_data = self.data[(col1 == v1) & (col2 == v2)]
                            if len(new_data) == 0:
                                gains[(col1.name, col2.name)] = 0
                                continue
                            if col1.dtype.name == 'object':
                                new_data = new_data.drop(col1.name, axis=1)
                            if col2.dtype.name == 'object':
                                new_data = new_data.drop(col2.name, axis=1)
                            nodo_posible.data = new_data
                            nodo_posible.value = (v1, v2)
                            gains[(col1.name, col2.name)] -= len(new_data)/total_data * nodo_posible.calculate_entropy()
                            split_info -= len(new_data)/total_data * np.log2(len(new_data)/total_data)
                    if split_info != 0:
                        gains[(col1.name, col2.name)] /= split_info
                    

            result = max(gains, key=gains.get)
            self.division = result
            self.es_categorico = (tipos[result[0]], tipos[result[1]])
            self.bin = [False, False]
            if not self.es_categorico[0]:
                self.bin[0] = bins[result[0]]
            if not self.es_categorico[1]:
                self.bin[1] = bins[result[1]]
            return result

    def bining(self, column, strategy='median', use_bin=False):
        """
        Recibe una columa que tiene datos numéricos  y retorna un DataFrame.
        Por defecto usa la mediana, pero puede especificarse usar la media
        """
        if use_bin:
            value = self.bin
            if type(value) == list:
                for i in range(2):
                    if self.division[i] == column.name:
                        value = value[i]
        else:
            if strategy == 'median':
                value = column.median()
            elif strategy == 'mean':
                value = column.mean()
            else:
                raise ValueError('Estrategia no soportada')
        column = column.apply(lambda x: 1 if x > value else 0)
        return column, value


In [3]:
# Código clase árbol
class DecisionTree:

    def __init__(self, cut_type, strategy, max_depth=None, min_data=None, ignore_cols=None):
        self.cut_type = cut_type
        self.strategy = strategy
        self.max_depth = max_depth
        self.min_data = min_data
        self.root = None
        self.ignore_cols = ignore_cols


    def fit(self, X, Y):
        for col in self.ignore_cols:
            X = X.drop(col, axis=1)
        X[Y.name] = Y
        self.root = Node(X, Y.name)
        por_chequear = [self.root]

        if self.cut_type == 'traditional':
            while por_chequear:
                node = por_chequear.pop(0)
                division = node.get_best_division(self.strategy)
                if not self.check_deep(node) or not self.check_min_data(node):
                    node.es_hoja = True
                    continue
                if division:
                    col = node.data[division]
                    if col.dtype.name != 'object':
                        col, bin = node.bining(col, self.strategy)
                    values = col.value_counts().to_dict()
                    child_nodes = []
                    for v in values:
                        new_data = node.data[col == v]
                        if col.dtype.name == 'object':
                            new_data = new_data.drop(col.name, axis=1)
                        child = Node(new_data, node.target, depth=node.depth + 1, value=v)
                        child_nodes.append(child)
                        por_chequear.append(child)
                    node.children = child_nodes

        else:
            while por_chequear:
                node = por_chequear.pop(0)
                divisiones = node.get_best_division(self.strategy, 'transversal')
                if not self.check_deep(node) or not self.check_min_data(node):
                    node.es_hoja = True
                    continue
                if divisiones:
                    values = []
                    cols = []
                    for col in divisiones:
                        col = node.data[col]
                        if col.dtype.name != 'object':
                            col, bin = node.bining(col, self.strategy)
                        cols.append(col)
                        values.append(col.value_counts().to_dict())

                    child_nodes = []
                    for v1 in values[0]:
                        for v2 in values[1]:
                            new_data = node.data[(cols[0] == v1) & (cols[1] == v2)]
                            if len(new_data) == 0:
                                continue
                            if cols[0].dtype.name == 'object':
                                new_data = new_data.drop(cols[0].name, axis=1)
                            if cols[1].dtype.name == 'object':
                                new_data = new_data.drop(cols[1].name, axis=1)
                            child = Node(new_data, node.target, depth=node.depth + 1, value=(v1, v2))
                            child_nodes.append(child)
                            por_chequear.append(child)
                    node.children = child_nodes

        return self.root


    def predict(self, X):
        """
        Predice la columna resultante a partir de los datos X
        """
        por_chequear = [self.root]
        predicted = {}
        self.root.predict_data = X
        if self.cut_type == 'traditional':
            while por_chequear:
                node = por_chequear.pop(0)
                if node.es_hoja:
                    for i in list(node.predict_data.index.values):
                        predicted[i] = node.answer
                else:
                    col = node.predict_data[node.division]
                    if col.dtype.name != 'object':
                        col, bin = node.bining(col, self.strategy, True)

                    for child in node.children:
                        child.predict_data = node.predict_data[col == child.value]
                        por_chequear.append(child)
        else:
            while por_chequear:
                node = por_chequear.pop(0)
                if node.es_hoja:
                    for i in list(node.predict_data.index.values):
                        predicted[i] = node.answer
                else:
                    col1 = node.predict_data[node.division[0]]
                    col2 = node.predict_data[node.division[1]]
                    if col1.dtype.name != 'object':
                        col1, bin = node.bining(col1, self.strategy, True)
                    if col2.dtype.name != 'object':
                        col2, bin = node.bining(col2, self.strategy, True)

                    for child in node.children:
                        child.predict_data = node.predict_data[(col1 == child.value[0]) & (col2 == child.value[1])]
                        por_chequear.append(child)
        return pd.Series(predicted).sort_index()


    def check_deep(self, node):
        """
        Retorna True o False dependiendo si el nodo puede seguir 
        su propagación en función de max_depth
        """
        if not self.max_depth:
            return True
        return node.depth < self.max_depth


    def check_min_data(self, node):
        """
        Retorna True o False dependiendo si el nodo puede seguir 
        su propagación en función de la cantidad mínima de datos
        """
        if not self.min_data:
            return True
        return len(node.data) >= self.min_data



## Comparar resultados

Antes de comparar los resultados, se cargan los datos y se define la función para obtener el rendimiento del árbol.

In [6]:
# En esta sección se encargan de abrir el dataset y dejarlo en formato X, Y para train y test.
df_train = pd.read_csv('datasets/train_small.csv')
X_train = df_train
Y_train = df_train['state']

df_test = pd.read_csv('datasets/test_small.csv')
X_test = df_test
Y_test= df_test['state']

In [7]:
def performance(y_real, y_predicted):
    total_data = len(y_predicted)
    correct = 0
    # Solo se tomará en cuenta las filas que sí están en el set de testeo
    for i in list(y_predicted.index.values):
        if y_real[i] == y_predicted[i]:
            correct += 1
    return correct/total_data


### Estrategia

Debido al gran tiempo que demora el árbol en generarse, se le entregó el hiper-parámetro `max_depth=3` en ambos árboles, de modo que solo se diferencien en el tipo de estrategia. 

In [8]:
# En esta sección se encargan de entrenar 2 árboles según distinto tipo de corte y luego comparar los resultados
# con el set de test
tree_median = DecisionTree('traditional', 'median', max_depth=3, ignore_cols=['name', 'ID', 'state'])
tree_mean = DecisionTree('traditional', 'mean', max_depth=3, ignore_cols=['name', 'ID', 'state'])

tree_median.fit(X_train, Y_train)
tree_mean.fit(X_train, Y_train)

Y_median = tree_median.predict(X_test)
Y_mean = tree_mean.predict(X_test)

In [9]:
print('Usando mediana se obtuvo un desempeño de', performance(Y_test, Y_median))
print('Usando media se obtuvo un desempeño de', performance(Y_test, Y_mean))

Usando mediana se obtuvo un desempeño de 0.6563626366913068
Usando media se obtuvo un desempeño de 0.6589453974389949


A partir de los resultados, obtenemos que el mejor desemeño fue con la estrategia de media, superando por aproximadamente 0.3% a la estrategia de mediana.

### Tipo de corte

In [10]:
# En esta sección se encargan de entrenar 2 árboles según estrategia y luego comparar los resultados
# con el set de test
tree_traditional = DecisionTree('traditional', 'mean', max_depth=3, ignore_cols=['name', 'ID', 'state'])
tree_transversal = DecisionTree('transversal', 'mean', max_depth=3, ignore_cols=['name', 'ID', 'state'])

tree_traditional.fit(X_train, Y_train)
tree_transversal.fit(X_train, Y_train)

Y_traditional = tree_traditional.predict(X_test)
Y_transversal = tree_transversal.predict(X_test)

In [11]:
print('Usando corte tradicional se obtuvo un desempeño de', performance(Y_test, Y_traditional))
print('Usando corte transversal se obtuvo un desempeño de', performance(Y_test, Y_transversal))

Usando corte tradicional se obtuvo un desempeño de 0.6589453974389949
Usando corte transversal se obtuvo un desempeño de 0.6442961510484203


A partir de los resultados, obtenemos que el mejor desemeño fue con el tipo de corte tradicional, superando por aproximadamente 1% al corte transversal.

### Hiperparámetros

#### Árbol con ambos hiper-parámetros como `None`
Para esta parte tuve enormes dificultades dado el tiempo que demora el árbol en generarse, por lo que tuve que usar al menos un hiper-parámetro para poder seguir avanzando con la tarea ☹️ .

Se tomará como árbol base el que posee máximo de profundidad de tres niveles.



In [12]:
# En esta sección se encargan de entrenar 4 árboles distintos y luego comparar los resultados
# con el set de test
tree = DecisionTree('traditional', 'mean', max_depth=3, ignore_cols=['name', 'ID', 'state'])
tree.fit(X_train, Y_train)
Y_predicted = tree.predict(X_test)
performance(Y_test, Y_predicted)

0.6589453974389949

### Árbol cambiando el hiper-parámetro `min_data`
Ahora se procede a cambiar el hiper-parámetro de mínimo de datos:

In [13]:
for i in [50, 100, 500, 1000]:
    tree = DecisionTree('traditional', 'mean', max_depth=3, min_data = i, ignore_cols=['name', 'ID', 'state'])
    tree.fit(X_train, Y_train)
    Y_predicted = tree.predict(X_test)
    print(f'Rendimiento con un mínimo de {i} datos:', performance(Y_test, Y_predicted))

Rendimiento con un mínimo de 50 datos: 0.6589453974389949
Rendimiento con un mínimo de 100 datos: 0.6589453974389949
Rendimiento con un mínimo de 500 datos: 0.6592637296318649
Rendimiento con un mínimo de 1000 datos: 0.6592637296318649


Se observa que al aumentar el mínimo de datos, el rendimiento aumenta ligeramente, y se mantiene estable desde 500 datos como mínimo.

Por lo tanto, el mejor valor para especificar como mínimo de datos es entre 500 y 1000. 




### Árbol cambiando el hiper-parámetro `max_depth`

Ahora se procede a cambiar el hiper-parámetro de máximo nivel de profundidad:



In [14]:
for i in [2, 3, 4]:
    tree = DecisionTree('traditional', 'mean', max_depth = i, ignore_cols=['name', 'ID', 'state'])
    tree.fit(X_train, Y_train)
    Y_predicted = tree.predict(X_test)
    print(f'Rendimiento con un máximo de {i} niveles de profundidad:', performance(Y_test, Y_predicted))

Rendimiento con un máximo de 2 niveles de profundidad: 0.5964060662706907
Rendimiento con un máximo de 3 niveles de profundidad: 0.6589453974389949
Rendimiento con un máximo de 4 niveles de profundidad: 0.6538950764744206


Se observa que con dos niveles de profundidad el rendimiento disminuye bastante (cerca de 10%), luego aumenta con tres niveles, y finalmente vuelve a disminuir con cuatro niveles.

Por lo tanto, el mejor valor para considerar como máximo de profundidad es 3.

### Árbol cambiando ambos hiper-parámetros

Finalmente, se preocede a cambiar ambos hiper-parámetros:

In [15]:
print('Rendimiento con:')
for i, j in [(3, 50), (3, 500), (3, 1000), (4, 500), (2, 500)]:
    tree = DecisionTree('traditional', 'mean', max_depth = i, min_data = j, ignore_cols=['name', 'ID', 'state'])
    tree.fit(X_train, Y_train)
    Y_predicted = tree.predict(X_test)
    performance(Y_test, Y_predicted)
    print(f'Máximo de {i} niveles de profundidad y mínimo de {j} datos:', performance(Y_test, Y_predicted))

Rendimiento con:
Máximo de 3 niveles de profundidad y mínimo de 50 datos: 0.6589453974389949
Máximo de 3 niveles de profundidad y mínimo de 500 datos: 0.6592637296318649
Máximo de 3 niveles de profundidad y mínimo de 1000 datos: 0.6592637296318649
Máximo de 4 niveles de profundidad y mínimo de 500 datos: 0.6589149631063264
Máximo de 2 niveles de profundidad y mínimo de 500 datos: 0.5964060662706907


Se observa que la combinación de hiper-parámetros con mejor rendimiento fue especificar 3 niveles de profundidad y entre 500 y 1000 datos mínimos, donde el rendimiento se mantuvo igual.

Finalmente, el desempeño del árbol cambió en mayor proporción al variar la profundidad máxima, teniendo tanto su peor desempeño (2 niveles) como su mejor desempeño (3 niveles).

## Visualización

Para visualizar el árbol, se hará uso de la librería `graphviz`, y en particular se usará la clase `Digraph`.

En cada nodo se especifica su división, número de muestras en el nodo y predicción. Si es un nodo hoja, en lugar de división se menciona que es hoja.

En cada arista de un nodo a su hijo, se especifica el valor que toma entre paréntesis. Si es un atributo contínuo, se especifica si es mayor (>), o menor o igual (<=) al valor usado para separarlos.

In [16]:
import graphviz

In [17]:
tree = DecisionTree('traditional', 'mean', max_depth=3, ignore_cols=['name', 'ID', 'state'])
tree.fit(X_train, Y_train)

In [18]:
def create_graph(tree):
    """
    Crea la visaulización del árbol con corte tradicional
    """
    g = graphviz.Digraph('G', filename='graph.gv')
    g.attr('node', shape='square')
    por_recorrer = [tree]
    nodes = {}
    index = 0
    nodes[tree] = index
    g.node(f'struct{index}', f'División: {tree.division}\nSamples: {len(tree.data)}\nPredicción: {tree.answer}')
    while por_recorrer:
        node = por_recorrer.pop(0)
        if node.es_hoja:
            continue
        for child in node.children:   
            index += 1  
            nodes[child] = index
            if not node.es_categorico:
                if child.value:
                    added = '> '
                else:
                    added = '<= '
                added += str(node.bin)
            else:
                added = child.value
            if not child.es_hoja:
                g.node(f'struct{index}', f'División: {child.division}\nSamples: {len(child.data)}\nPredicción: {child.answer}')
            else:
                g.node(f'struct{index}', f'División: Es hoja\nSamples: {len(child.data)}\nPredicción: {child.answer}')
            g.edge(f'struct{nodes[node]}', f'struct{index}', label=f'({added})')

        por_recorrer.extend(node.children)
    return g

In [ ]:
create_graph(tree.root)

## Bonus

In [20]:
tree = DecisionTree('transversal', 'mean', max_depth=3, ignore_cols=['name', 'ID', 'state'])
tree.fit(X_train, Y_train)

KeyboardInterrupt: 

In [ ]:
def create_graph_transversal(tree):
    """
    Crea la visaulización del árbol con corte transversal
    """
    g = graphviz.Digraph('G', filename='graph.gv')
    g.attr('node', shape='square')
    por_recorrer = [tree]
    nodes = {}
    index = 0
    nodes[tree] = index
    tree_label = ' & '.join(tree.division)
    g.node(f'struct{index}', f'División: {tree_label}\nSamples: {len(tree.data)}\nPredicción: {tree.answer}')
    while por_recorrer:
        node = por_recorrer.pop(0)
        if node.es_hoja:
            continue
        for child in node.children:   
            index += 1  
            nodes[child] = index
            if not node.es_categorico[0]:
                if child.value[0]:
                    added_1 = '> '
                else:
                    added_1 = '<= '
                added_1 += str(round(node.bin[0], 2))
            else:
                added_1 = child.value[0]
            if not node.es_categorico[1]:
                if child.value[1]:
                    added_2 = '> '
                else:
                    added_2 = '<= '
                added_2 += str(round(node.bin[1], 2))
            else:
                added_2 = child.value[1]
            label = f'{added_1} & {added_2}'
            added_1 + "&" + added_2 + f'{child.value[1]}'
            if not child.es_hoja:
                div_label = ' & '.join(child.division)
                g.node(f'struct{index}', f'División: {div_label}\nSamples: {len(child.data)}\nPredicción: {child.answer}')
            else:
                g.node(f'struct{index}', f'División: Es hoja\nSamples: {len(child.data)}\nPredicción: {child.answer}')
            g.edge(f'struct{nodes[node]}', f'struct{index}', label=f'({label})')

        por_recorrer.extend(node.children)
    return g


In [ ]:
create_graph_transversal(tree.root)